# ICD Project

## Importing libraries

In [7]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import CountVectorizer
import string

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/dani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/dani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/dani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Load Dataset

In [8]:
data = pd.read_csv('icd_scopus.csv')
data

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,NaN,NaN,NaN,International Conference on Applied Research i...,2023,Springer Proceedings in Business and Economics,NaN,NaN,NaN,NaN,...,978-981198484-6,NaN,NaN,English,Springer Proc. Bus. Econ.,Conference review,Final,NaN,Scopus,2-s2.0-85161683590
1,Vargas A.C.; Magnussen R.,"Vargas, Alejandra Celis (57951796800); Magnuss...",57951796800; 55579157200,A Game-based Approach for Open Data in Educati...,2022,Proceedings of the European Conference on Game...,2022-October,NaN,NaN,139,...,978-191458751-1,NaN,NaN,English,Proc. European Conf. Games-based Learn.,Conference paper,Final,NaN,Scopus,2-s2.0-85141152410
2,Fiş Erümit S.; Karakuş Yılmaz T.,"Fiş Erümit, S. (36343307300); Karakuş Yılmaz, ...",36343307300; 24166942000,Gamification Design in Education: What Might G...,2022,"Technology, Knowledge and Learning",27,4,NaN,1039,...,NaN,ICMLF,NaN,English,Tech. Knowl. Learn.,Article,Final,NaN,Scopus,2-s2.0-85128920438
3,Alsamawi F.N.; Kurnaz S.,"Alsamawi, Farazdaq Nahedh (57224682003); Kurna...",57224682003; 57203640255,A framework for adopting gamified learning sys...,2023,Applied Nanoscience (Switzerland),13,2,NaN,1135,...,NaN,NaN,NaN,English,Appl. Nanosci. (Switzerland),Article,Final,All Open Access; Bronze Open Access; Green Ope...,Scopus,2-s2.0-85108164717
4,Karnalim O.; Simon; Chivers W.,"Karnalim, Oscar (57191242240); Simon (74097108...",57191242240; 7409710825; 6505969230,Non-game Incentives in Gamified Programming Ed...,2023,Lecture Notes in Networks and Systems,633 LNNS,NaN,NaN,910,...,978-303126875-5,NaN,NaN,English,Lect. Notes Networks Syst.,Conference paper,Final,NaN,Scopus,2-s2.0-85151127571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,Unkelos-Shpigel N.,"Unkelos-Shpigel, Naomi (55786671200)",55786671200,Peel the onion: Use of collaborative and gamif...,2016,Lecture Notes in Business Information Processing,249,NaN,NaN,122,...,978-331939563-0,NaN,NaN,English,Lect. Notes Bus. Inf. Process.,Conference paper,Final,NaN,Scopus,2-s2.0-84976621686
378,Chou C.C.; He S.-J.; Olsen J.,"Chou, Chientzu Candace (56900344000); He, Shi-...",56900344000; 57190391124; 57200244427,The impact of badges on course participation a...,2015,Workshop Proceedings of the 23rd International...,NaN,NaN,NaN,333,...,978-499080147-2,NaN,NaN,English,"Workshop Proc. Int. Conf. Comput. Educ., ICCE",Conference paper,Final,NaN,Scopus,2-s2.0-85040455534
379,Zarzycki A.,"Zarzycki, Andrzej (33668016500)",33668016500,Teaching and Designing for Augmented Reality,2014,Proceedings of the International Conference on...,1,NaN,NaN,357,...,978-949120707-5,NaN,NaN,English,Proc. Int. Conf. Educ. Res. Comput. Aided. Arc...,Conference paper,Final,NaN,Scopus,2-s2.0-85006776989
380,Arias Aranda D.; Bustinza Sánchez O.F.; Djundu...,"Arias Aranda, Daniel (7801374665); Bustinza Sá...",7801374665; 35791444600; 57016876400,Effects of gamified business simulations on en...,2016,Revista de Educacion,2016,371,NaN,126,...,NaN,NaN,NaN,English,Revista Educ.,Article,Final,NaN,Scopus,2-s2.0-84951275541


## Basic Text pre-processing

### Checking for duplicate values

In [4]:
data.duplicated().sum()

0

Zero duplicated values, so there's no need to try to remove them.

### Cheking for null values


In [11]:
# Checking for missing values
data.isna().sum()

Authors                           18
Author full names                 18
Author(s) ID                      18
Title                              0
Year                               0
Source title                       0
Volume                           103
Issue                            223
Art. No.                         296
Page start                       108
Page end                         112
Page count                        98
Cited by                           0
DOI                               74
Link                               0
Affiliations                      20
Authors with affiliations         20
Abstract                           0
Author Keywords                   41
Index Keywords                   150
Molecular Sequence Numbers       382
Chemicals/CAS                    378
Tradenames                       381
Manufacturers                    382
Funding Details                  278
Funding Texts                    280
References                        24
C

### Remove useless columns in the context and columns with NaN values

In [19]:
columns_to_drop = ['Author full names', 'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end', 'Page count', 'Cited by', 'DOI', 'Link', 'Affiliations', 'Authors with affiliations', 'Sponsors', 'Molecular Sequence Numbers', 'Chemicals/CAS', 'Tradenames', 'Manufacturers', 'ISBN', 'CODEN', 'PubMed ID']

data = data.drop(columns=columns_to_drop)
data

KeyError: "['Author full names', 'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end', 'Page count', 'Cited by', 'DOI', 'Link', 'Affiliations', 'Authors with affiliations', 'Sponsors', 'Molecular Sequence Numbers', 'Chemicals/CAS', 'Tradenames', 'Manufacturers', 'ISBN', 'CODEN', 'PubMed ID'] not found in axis"

## **2. Preprocess the dataset**
